# Quick start demo
## Install packages

In [ ]:

#r "nuget:Newtonsoft.Json,12.03.3"
#r "JsonSubTypes,1.6.0"
#r "nuget:RestSharp,105.1.0"
#r "nuget:Polly,7.2.2"
#r "nuget:TileDB.CSharp,2.4.7"
#r "nuget:TileDB.Cloud"

Restore sources C:\Nuget.Local Installed Packages RestSharp, 105.1.0 TileDB.Cloud, 0.1.0

In [ ]:
TileDB.Cloud.Client.Login()

In [ ]:
var user_details = TileDB.Cloud.RestUtil.GetUser();
System.Console.WriteLine("{0}", user_details.ToJson());

{
  "id": "ca55fd8a-5243-46f9-b7d5-7cd5e3d2b889",
  "username": "bin.deng",
  "name": "Bin Deng",
  "email": "bin.deng@tiledb.com",
  "is_valid_email": true,
  "organizations": [
    {
      "role": "read_only",
      "user_id": "ca55fd8a-5243-46f9-b7d5-7cd5e3d2b889",
      "organization_id": "bbacb27e-f973-446f-a72a-3b104a9e4932",
      "username": "bin.deng",
      "organization_name": "TileDB-Inc"
    },
    {
      "role": "read_only",
      "user_id": "ca55fd8a-5243-46f9-b7d5-7cd5e3d2b889",
      "organization_id": "df9332da-82f1-49c2-b913-7964d250002f",
      "username": "bin.deng",
      "organization_name": "public"
    }
  ],
  "enabled_features": [
    "notebooks",
    "generic_udfs",
    "notebook_sharing"
  ]
}
